In [10]:
## this csv file lies in kaggle folder in drive google
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/kaggle/shortjokes.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kaggle/')

In [16]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [14]:
pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/kaggle/shortjokes.csv')
# df[df['Topic']!='0']

In [2]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[:10000])
dataset

Dataset({
    features: ['ID', 'Joke'],
    num_rows: 10000
})

In [3]:
def add_prefix(examples):

    prompt_prefix='find important topics from the text. \n Text: '
    prompt_suffix='\n Topics: '
    examples1 = [prompt_prefix+ example['Joke']+prompt_suffix for example in examples]
    return examples1

prompts = add_prefix(dataset)


In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install tqdm

In [ ]:

from transformers import pipeline, AutoTokenizer
import torch

# torch.manual_seed(0)
model = "tiiuae/falcon-7b-instruct"

# tokenizer = AutoTokenizer.from_pretrained(model)
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     device='cuda',
# )
# torch.manual_seed(2)

from tqdm.auto import tqdm
# prompt_prefix = 'find important topics from the text. \n Text: '
# prompt_suffix = '\n Topics: '
results = {}
index=0
for prompt in tqdm(prompts, total=len(prompts)):
    # prompto = prompt_prefix + prompt + prompt_suffix

    sequences = pipe(
        prompt,
        max_new_tokens=25,
        return_full_text = False,
    )

    if len(sequences)<1:
        results[index]='0'
    else:
        results[index]=[]
        for seq in sequences:
            results[index].append(seq)
            # print(f"{seq['generated_text']}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token

In [60]:
df_results=pd.DataFrame(results)
df_results.to_csv('/content/drive/MyDrive/kaggle/results.csv')

[narrators, documentaries]


In [46]:
prompts[1]

'Return a list of named entities in the text. \n Text: Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men.\nNamed entities: '

In [ ]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# model=AutoModelForTokenClassification.from_pretrained("ml6team/keyphrase-extraction-kbir-inspec")
# model= model.to('cuda')

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        return np.unique([result.get("word").strip() for result in results])


In [ ]:
# Load pipeline
model_name = "ml6team/keyphrase-extraction-kbir-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name,device='cuda')


In [ ]:
# !pip install datasets
from datasets import Dataset
dataset = Dataset.from_pandas(df[:10000])
dataset

Dataset({
    features: ['ID', 'Joke'],
    num_rows: 10000
})

In [ ]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
arr=list()

for topics in tqdm(extractor(KeyDataset(dataset, "Joke"), batch_size=64), total=len(df[:10000])):
    # topics = extractor(elements)
    if len(topics)<2:
        arr.append('0')
    else:
        arr.append(topics)
df_new = pd.DataFrame()
df_new['Topic'] = arr
df_new['Joke'] = df['Joke'][:10000]
df_new.to_csv('/content/drive/MyDrive/kaggle/joke_topics.csv')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
df_new

,Topic,Joke
0,0,"[me narrating a documentary about narrators] ""..."
1,0,Telling my daughter garlic is good for you. Go...
2,0,I've been going through a really rough period ...
3,0,"If I could have dinner with anyone, dead or al..."
4,0,Two guys walk into a bar. The third guy ducks.
...,...,...
9995,"[porcupine, sheep]",Q: What do you get when you cross a sheep and ...
9996,0,Son: can I get lunch money Dad: I have a boyfr...
9997,"[Helen Keller, plate Glass window]",Helen Keller walks into a bar... And then a ta...
9998,"[Nun, Priest, Rabbi, bartender]","A Nun, a Priest, and a Rabbi walk into a bar....."


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


100


KeyboardInterrupt: ignored

,Joke,Topic
0,"[me narrating a documentary about narrators] ""...",narrators
1,Telling my daughter garlic is good for you. Go...,immune system
2,I've been going through a really rough period ...,sand paper
3,"If I could have dinner with anyone, dead or al...",[]
4,Two guys walk into a bar. The third guy ducks.,[]
...,...,...
195,I went in to hospital for an operation... I as...,anaesthetist
196,A piece of shit walks into a bar It's my dad.....,[]
197,What's a snakes favourite dance ? The mamba !,mamba
198,What do you call a cheap circumcision? A rip-off!,cheap circumcision


"A piece of shit walks into a bar It's my dad... My dad is a piece of shit."